# Following [SpaGE Tutorial](https://github.com/tabdelaal/SpaGE/blob/master/SpaGE_Tutorial.ipynb)

Integration of *osmFISH* spatial data with the *AllenSSp* scRNA-seq data

In [1]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt

import scipy
import scipy.stats as st
from scipy import linalg
from scipy import sparse as sp
import pickle
from sklearn.metrics.pairwise import euclidean_distances

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/Volumes/LaCie/school/combine_lab/SpaGE/SpaGE_Datasets/'

# import spatial data

In [3]:
ds = loompy.connect(path + 'Spatial/osmFISH/osmFISH_SScortex_mouse_all_cells.loom')
FISH_Genes = ds.ra['Gene']   
colAtr = ds.ca.keys()

df = pd.DataFrame()
for i in colAtr:
    df[i] = ds.ca[i]

osmFISH_meta = df.iloc[np.where(df.Valid == 1)[0], :]
osmFISH_data = ds[:,:]
osmFISH_data = osmFISH_data[:,np.where(df.Valid == 1)[0]]
osmFISH_data = pd.DataFrame(data= osmFISH_data, index= FISH_Genes)

del ds, colAtr, i, df, FISH_Genes

# Select cortical regions only to match the AllenSSp dataset
Cortex_Regions = ['Layer 2-3 lateral', 'Layer 2-3 medial', 'Layer 3-4', 
                  'Layer 4','Layer 5', 'Layer 6', 'Pia Layer 1']
Cortical = np.stack(i in Cortex_Regions for i in osmFISH_meta.Region)

osmFISH_meta = osmFISH_meta.iloc[Cortical,:]
osmFISH_data = osmFISH_data.iloc[:,Cortical]
del Cortex_Regions,Cortical

In [4]:
osmFISH_meta

,CellID,ClusterID,ClusterName,Region,Total_molecules,Valid,X,Y,_tSNE_1,_tSNE_2,size_pix,size_um2
0,778,18,Inhibitory CP,Layer 6,390,1,18171.230942,24590.795275,-58.132385,5.219181,60911.0,257.348975
2,3642,18,Inhibitory CP,Layer 4,405,1,11247.433715,36626.892415,-61.572853,-1.185364,44143.0,186.504175
5,769,18,Inhibitory CP,Layer 4,491,1,18374.295796,26286.856654,-66.862244,-1.766500,41014.0,173.284150
7,5870,18,Inhibitory CP,Layer 4,518,1,4197.789280,39465.355903,-59.923431,-3.739808,42376.0,179.038600
9,5682,18,Inhibitory CP,Layer 2-3 medial,210,1,3757.866349,46281.167166,-64.769470,5.166343,23922.0,101.070450
...,...,...,...,...,...,...,...,...,...,...,...,...
4832,3635,25,Vascular Smooth Muscle,Layer 3-4,59,1,11457.806019,38036.523162,-12.763592,7.068469,7993.0,33.770425
4833,3336,25,Vascular Smooth Muscle,Pia Layer 1,144,1,9214.047907,45314.585476,0.060634,30.242371,14865.0,62.804625
4834,6462,25,Vascular Smooth Muscle,Layer 6,236,1,4476.301656,31097.786260,3.578825,28.862700,24835.0,104.927875
4835,260,25,Vascular Smooth Muscle,Layer 2-3 lateral,67,1,18796.948664,36603.475262,-11.011883,5.821001,8873.0,37.488425


In [5]:
# Normalization of ST gene expression
cell_count = np.sum(osmFISH_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)

osmFISH_data_norm = osmFISH_data.apply(Log_Norm_spatial,axis=0)

# scale
genes = st.zscore(osmFISH_data_norm)

In [7]:
cell_st = osmFISH_meta.loc[:,['CellID', 'X', 'Y']]

# add gene expression info for each cell at spot i
cell_spot = cell_st.merge(genes.T, left_index=True, right_index=True, how='left')

cell_spt = cell_spot.reset_index(drop=True) # reset index
cell_spt

,CellID,X,Y,Gad2,Slc32a1,Crhbp,Cnr1,Vip,Cpne5,Pthlh,...,Ctps,Anln,Mrc1,Hexb,Ttr,Foxj1,Vtn,Flt1,Apln,Acta2
0,778,18171.230942,24590.795275,1.621371,1.777401,-1.285498,0.169445,-0.417264,0.932325,-1.285498,...,0.817961,0.050672,-0.237032,0.876704,-0.237032,-0.417264,-0.635124,-1.285498,-0.237032,-0.417264
1,3642,11247.433715,36626.892415,2.723553,2.088070,0.162081,-0.609545,-0.353881,-0.609545,-0.609545,...,0.162081,-0.150690,-0.609545,-0.150690,-0.609545,-0.609545,-0.353881,-0.954585,0.017943,-0.609545
2,769,18374.295796,26286.856654,2.459380,2.289477,1.116087,-0.466125,-0.148632,-0.951611,0.186618,...,-0.022988,-0.148632,-0.293912,1.150048,-0.677581,-0.148632,-0.022988,-0.951611,-0.466125,-0.293912
3,5870,4197.789280,39465.355903,2.253989,1.549556,-0.197524,0.225092,-0.510714,-0.982660,0.135831,...,0.225092,-0.982660,-0.072863,0.515976,-0.072863,-0.717495,-0.717495,-0.982660,-0.717495,-0.072863
4,5682,3757.866349,46281.167166,1.496840,2.316222,-0.944582,1.345124,0.013995,-0.944582,-0.944582,...,0.495107,-0.354909,0.013995,0.283120,-0.354909,-0.354909,-0.354909,-0.354909,0.283120,-0.944582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,3635,11457.806019,38036.523162,0.309345,-0.843207,-0.843207,-0.843207,0.309345,0.309345,-0.843207,...,1.309683,-0.843207,1.086314,-0.843207,-0.843207,-0.843207,-0.843207,-0.843207,0.309345,1.086314
3401,3336,9214.047907,45314.585476,1.396286,-0.977202,-0.977202,1.229384,1.229384,-0.977202,-0.977202,...,0.381686,0.759097,0.110551,-0.278649,-0.278649,-0.977202,1.024502,0.381686,-0.278649,2.598400
3402,6462,4476.301656,31097.786260,0.232061,-0.423666,-1.342403,-0.423666,-0.156431,-1.342403,-0.423666,...,0.055889,0.232061,2.117512,-0.784520,0.830838,0.232061,-0.423666,-1.342403,0.232061,1.848433
3403,260,18796.948664,36603.475262,0.632525,-1.041085,-1.041085,1.198118,0.632525,0.129437,-1.041085,...,0.129437,0.129437,-1.041085,-1.041085,0.129437,-1.041085,-1.041085,-1.041085,0.632525,1.389190


# Get ddf(x) for all training samples

## Create adj mat (inspired by [GCNG](https://github.com/xiaoyeye/GCNG/blob/master/data_generation_interaction_ten_fold.py), [SpaGCN](https://github.com/jianhuupenn/SpaGCN), and [SpatialDE](https://github.com/Teichlab/SpatialDE))


From [GCNG](https://github.com/xiaoyeye/GCNG/blob/2fba06bac64e97744a9a8d2a1505aa5c2117730c/data_generation_interaction_ten_fold.py#L19)

In [10]:
#### from GCNG ####

############ the distribution of distance
distance_list = []
print ('calculating distance matrix, it takes a while')

# compute euclidean distance, considering both physical location and gene expression
for j in range(cell_st.shape[0]):
    for i in range(cell_st.shape[0]):
        if i!=j:
            distance_list.append(np.linalg.norm(cell_spot.iloc[j, 1:]-cell_spot.iloc[i, 1:]))

# save out file
np.save(path+'dist_array_eucl_xyz.npy',np.array(distance_list))

# np.savetxt("dist_list.csv", 
#            distance_list,
#            delimiter ="\t")

calculating distance matrix, it takes a while


In [15]:
distance_array = np.array(distance_list)
distance_array.shape

(11590620,)

import saved distance array (instead of re-running since it takes a long time to compute)

In [ ]:
distance_array = np.load(path+'dist_array_eucl_xyz.npy')
print(distance_array.shape)
distance_array

from [spektral](https://github.com/danielegrattarola/spektral/blob/1dbdf9e919ff4606ee5375567a9b993687c95bcd/spektral/utils/convolution.py#L47) -- unable to import the package (received 'illegal hardware' error)

In [12]:
### from spektral ###
def degree_power(A, k):
    r"""
    Computes \(\D^{k}\) from the given adjacency matrix. Useful for computing
    normalised Laplacian.
    :param A: rank 2 array or sparse matrix.
    :param k: exponent to which elevate the degree matrix.
    :return: if A is a dense array, a dense array; if A is sparse, a sparse
    matrix in DIA format.
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        degrees = np.power(np.array(A.sum(1)), k).ravel()
    degrees[np.isinf(degrees)] = 0.0
    if sp.issparse(A):
        D = sp.diags(degrees)
    else:
        D = np.diag(degrees)
    return D

def normalized_adjacency(A, symmetric=True):
    r"""
    Normalizes the given adjacency matrix using the degree matrix as either
    \(\D^{-1}\A\) or \(\D^{-1/2}\A\D^{-1/2}\) (symmetric normalization).
    :param A: rank 2 array or sparse matrix;
    :param symmetric: boolean, compute symmetric normalization;
    :return: the normalized adjacency matrix.
    
    motivation: Takes into account both number of neighbours and their own number of neighbours
    """
    if symmetric:
        normalized_D = degree_power(A, -0.5)
        return normalized_D.dot(A).dot(normalized_D)
    else:
        normalized_D = degree_power(A, -1.0)
        return normalized_D.dot(A)

In [29]:
distance_matrix = euclidean_distances(cell_spot.iloc[:, 1:], cell_spot.iloc[:, 1:])
unweighted_Adj = np.zeros(distance_matrix.shape) # initialize distance matrix to zero

for i in range(unweighted_Adj.shape[0]):
    for j in range(unweighted_Adj.shape[1]):
        if distance_matrix[i,j] > 0:
            unweighted_Adj[i,j] = 1
unweighted_Adj

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [27]:
############### get normalized sparse adjacent matrix
distance_matrix_threshold_I_N = normalized_adjacency(unweighted_Adj, symmetric=True)
# distance_matrix_threshold_I_N = np.float32(whole_distance_matrix_threshold_I) ## do not normalize adjcent matrix
distance_matrix_threshold_I_N = np.float32(unweighted_Adj)
distance_matrix_threshold_I_N_crs = sp.csr_matrix(distance_matrix_threshold_I_N)
with open(path+'unweighted_adj_mat_crs.pickle', 'wb') as fp:
    pickle.dump(distance_matrix_threshold_I_N_crs, fp)

In [28]:
distance_matrix_threshold_I_N

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]], dtype=float32)

In [ ]:
###try different distance threshold, so that on average, each cell has x neighbor cells, see Tab. S1 for results
for threshold in range(100,400,40): #[100,140,180,210,220,260]:#range (210,211):#(100,400,40):
    num_big = np.where(distance_array<threshold)[0].shape[0]
    print (threshold,num_big,str(num_big/(len(cell_st)*2)))
    distance_matrix_threshold_I_list = []
#     distance_matrix_threshold_W_list = []
    distance_matrix = euclidean_distances(cell_st[['X','Y']], cell_st[['X','Y']])
    distance_matrix_threshold_I = np.zeros(distance_matrix.shape)
#     distance_matrix_threshold_W = np.zeros(distance_matrix.shape)
    for i in range(distance_matrix_threshold_I.shape[0]):
        for j in range(distance_matrix_threshold_I.shape[1]):
            if distance_matrix[i,j] <= threshold and distance_matrix[i,j] > 0:
                distance_matrix_threshold_I[i,j] = 1
#                 distance_matrix_threshold_W[i,j] = distance_matrix[i,j]
    distance_matrix_threshold_I_list.append(distance_matrix_threshold_I)
#     distance_matrix_threshold_W_list.append(distance_matrix_threshold_W)
# whole_distance_matrix_threshold_I = scipy.linalg.block_diag(distance_matrix_threshold_I_list[0],
#                                                             distance_matrix_threshold_I_list[1],
#                                                             distance_matrix_threshold_I_list[2],
#                                                             distance_matrix_threshold_I_list[3],
#                                                             distance_matrix_threshold_I_list[4],
#                                                             distance_matrix_threshold_I_list[5],
#                                                             distance_matrix_threshold_I_list[6])
    ############### get normalized sparse adjacent matrix
    distance_matrix_threshold_I_N = normalized_adjacency(distance_matrix_threshold_I, symmetric=True)
    # distance_matrix_threshold_I_N = np.float32(whole_distance_matrix_threshold_I) ## do not normalize adjcent matrix
    distance_matrix_threshold_I_N = np.float32(distance_matrix_threshold_I)
    distance_matrix_threshold_I_N_crs = sp.csr_matrix(distance_matrix_threshold_I_N)
    with open(path+'adj_mat_crs_'+str(threshold)+'.pickle', 'wb') as fp:
        pickle.dump(distance_matrix_threshold_I_N_crs, fp)

Import saved Adjacency matrices:

In [ ]:
import os 

A = []
# read in all pickle files
for i, file in enumerate([path for path in os.listdir(path) if path.endswith('pickle')]):
    print(i, file)
    for i in [path+file]:
        with open(i, 'rb') as filepath:
            while True:
                try:
                    A.append(pickle.load(filepath))
                except EOFError:
                    break    
                    
# uncompress Compressed Sparse Row matrix
adj_mat = []
for i in range(len(A)):
    adj_mat.append(A[i].todense())

# get number of counts for all connected cells of cell i
sum_neighb_cell = []
for file in range(len(adj_mat)):
     sum_neighb_cell.append([(adj_mat[file][i][np.where(adj_mat[file][i] != 0)].size) for i in range(len(adj_mat[file]))])

# get average cell neighbourhood by threshold
for i in range(len(adj_mat)):
    avg_cell_neib = sum(sum_neighb_cell[i])/len(adj_mat[0])
    print('average cell neighborhood for file', i,':', avg_cell_neib)

GNCG [supp](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02214-w#Sec16)

seqFISH+ avg cell neighbourhood for different distance threshold:
- 100: 1.2
- 140: 2.5 <-- threshold with the best validation performace
- 180: 4.0
- 220: 5.8

MERFISH:
- 130: 1.03 <-- threshold with best validation performance
- 150: 1.52
- 170: 2.05
- 200: 2.95
- 240: 4.25

Try thresholds:
- 220: 1.12
- 300: 2.42

In [ ]:
A200 = adj_mat[4]
A300 = adj_mat[6]